# Segmenting and clustering neighbourhoods in Toronto

## Libraries

In [67]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import geocoder
import requests
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

## Task1

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
df = pd.read_html(url, header=0)[0]

In [4]:
df = df[df.Borough != "Not assigned"].reset_index(drop=True)

In [5]:
df = df.replace(r'/',', ', regex=True)

In [6]:
for index, row in df.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]

In [7]:
df.head(2)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village


In [8]:
df.shape

(103, 3)

## Task2

### Built a dataframe of the postal code of each neighborhood along with the borough name, neighborhood name,latitude and longitude coordinates of each neighborhood. 

#### Neighbourhoods coordonates

In [9]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.head(2)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [10]:
df2 = pd.merge(df, dfgeo, on="Postal Code", how='left')
df2.head(2)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572


## Task3

### Foursquare credentials

In [11]:
CLIENT_ID = 'LMPNBTTXPCFQSDJFXS0IJ4YH3HMZ1LREFCGVLW0Y0DCPGXT3' # your Foursquare ID
CLIENT_SECRET = 'RBTSAQ3JGTOEBGGNXIYRYOBVGX1BVFOBNXOFLSBJVRKZY3N2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: LMPNBTTXPCFQSDJFXS0IJ4YH3HMZ1LREFCGVLW0Y0DCPGXT3
CLIENT_SECRET:RBTSAQ3JGTOEBGGNXIYRYOBVGX1BVFOBNXOFLSBJVRKZY3N2


## Exploration of the first neighbourhoods of the dataframe

In [12]:
df2.loc[0, 'Neighbourhood']

'Parkwoods'

### Top 100 venues in Parkwoods within 500m radius

In [13]:
neighbourhood_latitude = df2.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df2.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df2.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [14]:
radius = 500
LIMIT = 100 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=LMPNBTTXPCFQSDJFXS0IJ4YH3HMZ1LREFCGVLW0Y0DCPGXT3&client_secret=RBTSAQ3JGTOEBGGNXIYRYOBVGX1BVFOBNXOFLSBJVRKZY3N2&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6021035cdb8c8f201f645b68'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA'

#### Extracting the catagory of the venues

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a dataframe.

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,649 Variety,Convenience Store,43.754513,-79.331942
2,Variety Store,Food & Drink Shop,43.751974,-79.333114
3,Corrosion Service Company Limited,Construction & Landscaping,43.752432,-79.334661


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Exploration of Totonto's neighbourhoods

#### This function repeats the same process to all the neighbourhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### The function is run to get the venue's, name, location and category for all the neighbourhoods in Toronto

In [20]:
toronto_venues = getNearbyVenues(names=df2['Borough'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude'])

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(2107, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


#### How many venues were returned for each neibourhood

In [22]:
toronto_venues.groupby('Neighbourhood').count().head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Central Toronto,108,108,108,108,108,108
Downtown Toronto,1212,1212,1212,1212,1212,1212
East Toronto,117,117,117,117,117,117
East York,75,75,75,75,75,75
Etobicoke,74,74,74,74,74,74


#### How many unique venues were returned for each neibourhood

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


## Analysis of each neighbourhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(2107, 272)

#### Rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.009259,...,0.000000,0.00000,0.000000,0.009259,0.000000,0.000000,0.000000,0.000000,0.0,0.009259
1,Downtown Toronto,0.0,0.000825,0.000825,0.000825,0.000825,0.00165,0.002475,0.000825,0.014026,...,0.011551,0.00165,0.000000,0.003300,0.000000,0.006601,0.000825,0.000000,0.0,0.005776
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.025641,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.017094
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,0.0,0.013333
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.013514,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.0,0.000000


In [27]:
toronto_grouped.shape

(10, 272)

#### Each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06
2     Pizza Place  0.06
3  Sandwich Place  0.06
4            Park  0.06


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.05
2           Restaurant  0.03
3  Japanese Restaurant  0.03
4                Hotel  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1    Greek Restaurant  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4         Pizza Place  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.07
1                 Park  0.05
2                 Bank  0.05
3  Sporting Goods Shop  0.04
4         Burger Joint  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1     Coffee Shop  0.05
2        Pharmacy  0.05
3  Sandwich Place  0.05
4             Gym  0.04


----Mississauga----
                       venue  freq
0                Coffee S

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Sushi Restaurant,Restaurant,Clothing Store,Dessert Shop,Pharmacy
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Pizza Place,Bakery,Café,Pub
3,East York,Coffee Shop,Park,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Supermarket,Grocery Store,Furniture / Home Store
4,Etobicoke,Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint


## Analysis of each neighbourhood

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:10]

In [32]:
print(toronto_grouped.shape)
print(neighbourhoods_venues_sorted.shape)
print(df2.shape)
print(kmeans.labels_.size)

(10, 272)
(10, 11)
(103, 5)
10


In [33]:
# add clustering labels
df3 = df2.copy()
df3.head(2)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572


In [34]:
df3 = df3.drop(["Postal Code"], axis = 1)
df3.head(2)

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572


In [59]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df3
# toronto_merged["Cluster Labels"] = kmeans.labels_
# merge toronto grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Borough')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
1,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym


In [60]:
toronto_merged

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
1,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,4,Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
99,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
100,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0,Coffee Shop,Greek Restaurant,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Pizza Place,Bakery,Café,Pub
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,4,Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint


In [68]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Examine clusters

### Cluster1

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
1,Victoria Village,Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
2,"Regent Park, Harbourfront",Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
3,"Lawrence Manor, Lawrence Heights",Coffee Shop,Clothing Store,Park,Restaurant,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Grocery Store
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
...,...,...,...,...,...,...,...,...,...,...,...
92,Stn A PO Boxes,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
96,"St. James Town, Cabbagetown",Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
97,"First Canadian Place, Underground city",Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym
99,Church and Wellesley,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant,Gym


### Cluster2

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Gas Station,Middle Eastern Restaurant,Burrito Place,Food Truck,Fried Chicken Joint,Sandwich Place,Intersection,Mediterranean Restaurant


### Cluster3

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Malvern, Rouge",Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bank,Bakery,Indian Restaurant,Pizza Place,Breakfast Spot,Thai Restaurant,Playground
8,"Parkview Hill, Woodbine Gardens",Coffee Shop,Park,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Supermarket,Grocery Store,Furniture / Home Store
12,"Rouge Hill, Port Union, Highland Creek",Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bank,Bakery,Indian Restaurant,Pizza Place,Breakfast Spot,Thai Restaurant,Playground
14,Woodbine Heights,Coffee Shop,Park,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Supermarket,Grocery Store,Furniture / Home Store
18,"Guildwood, Morningside, West Hill",Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bank,Bakery,Indian Restaurant,Pizza Place,Breakfast Spot,Thai Restaurant,Playground
22,Woburn,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bank,Bakery,Indian Restaurant,Pizza Place,Breakfast Spot,Thai Restaurant,Playground
23,Leaside,Coffee Shop,Park,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Supermarket,Grocery Store,Furniture / Home Store
26,Cedarbrae,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bank,Bakery,Indian Restaurant,Pizza Place,Breakfast Spot,Thai Restaurant,Playground
29,Thorncliffe Park,Coffee Shop,Park,Bank,Sporting Goods Shop,Burger Joint,Sandwich Place,Pizza Place,Supermarket,Grocery Store,Furniture / Home Store
32,Scarborough Village,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bank,Bakery,Indian Restaurant,Pizza Place,Breakfast Spot,Thai Restaurant,Playground


### Cluster4

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Humewood-Cedarvale,Convenience Store,Park,Field,Grocery Store,Discount Store,Jewelry Store,Breakfast Spot,Bus Line,Trail,Coffee Shop
21,Caledonia-Fairbanks,Convenience Store,Park,Field,Grocery Store,Discount Store,Jewelry Store,Breakfast Spot,Bus Line,Trail,Coffee Shop
56,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Convenience Store,Park,Field,Grocery Store,Discount Store,Jewelry Store,Breakfast Spot,Bus Line,Trail,Coffee Shop
63,"Runnymede, The Junction North",Convenience Store,Park,Field,Grocery Store,Discount Store,Jewelry Store,Breakfast Spot,Bus Line,Trail,Coffee Shop
64,Weston,Convenience Store,Park,Field,Grocery Store,Discount Store,Jewelry Store,Breakfast Spot,Bus Line,Trail,Coffee Shop


### Cluster5

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Islington Avenue, Humber Valley Village",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
11,"West Deane Park, Princess Gardens, Martin Grov...",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
70,Westmount,Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
77,"Kingsview Village, St. Phillips, Martin Grove ...",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
88,"New Toronto, Mimico South, Humber Bay Shores",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
89,"South Steeles, Silverstone, Humbergate, Jamest...",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
93,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
94,"Northwest, West Humber - Clairville",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
98,"The Kingsway, Montgomery Road, Old Mill North",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Gym,Bakery,Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint
